In [1]:
import os


In [4]:
files = {}
for f in os.listdir("data"):
    if ".jp2" in f:
        parts = f.split("_")
        name = "_".join(parts[:2])
        if name not in files:
            files[name] = {}
        path = f"data/{f}"
        files[name][parts[2]] = [path, os.path.getmtime(path)]


In [5]:
files


{'T34UFG_20150910T092726': {'B02': ['data/T34UFG_20150910T092726_B02_10m.jp2',
   1740564545.700425],
  'B03': ['data/T34UFG_20150910T092726_B03_10m.jp2', 1740564547.8594377],
  'B04': ['data/T34UFG_20150910T092726_B04_10m.jp2', 1740564550.5274532],
  'B08': ['data/T34UFG_20150910T092726_B08_10m.jp2', 1740564553.585471],
  'SCL': ['data/T34UFG_20150910T092726_SCL_20m.jp2', 1740564554.103474]},
 'T35VLE_20150824T094006': {'B02': ['data/T35VLE_20150824T094006_B02_10m.jp2',
   1740564550.654454],
  'B03': ['data/T35VLE_20150824T094006_B03_10m.jp2', 1740564566.7405477],
  'B04': ['data/T35VLE_20150824T094006_B04_10m.jp2', 1740564582.8726416],
  'B08': ['data/T35VLE_20150824T094006_B08_10m.jp2', 1740564604.3907669],
  'SCL': ['data/T35VLE_20150824T094006_SCL_20m.jp2', 1740564605.434773]},
 'T35UMB_20150811T093006': {'B02': ['data/T35UMB_20150811T093006_B02_10m.jp2',
   1740564555.0464795],
  'B03': ['data/T35UMB_20150811T093006_B03_10m.jp2', 1740564572.473581],
  'B04': ['data/T35UMB_201508